<a href="https://colab.research.google.com/github/shubham-ai/ChatGPT/blob/main/Sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install "langchain>=0.0.218" openai -qqq
!pip install "langchain"  --user

# assert langchain.__version__ >= "0.0.218", "Please ensure you are using LangChain v0.0.188 or higher"
import os
!pip install transformers[torch]
!pip install sentencepiece --user
!pip install einops --user
os.environ["Langchain"] = "langchain-testing"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00


In [2]:

!git lfs install
!git clone https://huggingface.co/google/flan-t5-base
# !git clone https://huggingface.co/google/flan-t5-xxl


Git LFS initialized.
Cloning into 'flan-t5-base'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 58 (delta 0), reused 0 (delta 0), pack-reused 55
Unpacking objects: 100% (58/58), 621.24 KiB | 4.78 MiB/s, done.
Filtering content: 100% (5/5), 3.87 GiB | 52.93 MiB/s, done.


In [52]:
import langchain
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
import datetime
import os
import torch
# turn on wandb logging for langchain
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained("flan-t5-base")

# model = T5ForConditionalGeneration.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto", torch_dtype=torch.float16).to(device)

pipe = pipeline(
    "text2text-generation",
    model=model, tokenizer=tokenizer, max_length=512
)

local_llm = HuggingFacePipeline(pipeline=pipe)

# llm = OpenAI(model_name="text-davinci-003", openai_api_key=API_KEY)

# first step in chain
start_time_ms = datetime.datetime.now().timestamp() * 1000

template='''You analyse the sentiment of the text provided.
The sentiment can be positive, negative or neutral.
{text}
'''
prompt_1 = PromptTemplate(

input_variables=["text"],

template=template)

chain_1 = LLMChain(llm = local_llm, prompt = prompt_1, verbose=False,output_key="sentiment")

# second step in chain

prompt_2 = PromptTemplate(

input_variables=["text"],

template='''tell me the percentage of politeness in the text provided. {text} ''')

chain_2 = LLMChain(llm = local_llm, prompt = prompt_2, verbose=False,output_key="politeness score")

# Third step in chain

prompt_3 = PromptTemplate(

input_variables=["text"],

template='''tell me only negative words found in the text provide. {text} ''')

chain_3 = LLMChain(llm = local_llm, prompt = prompt_3, verbose=False,output_key="negative words")
# fourth step in chain

prompt_4 = PromptTemplate(

input_variables=["text"],

template='''How many words does {text} have? ''')

chain_4 = LLMChain(llm = local_llm, prompt = prompt_4, verbose=False,output_key="words")
overall_chain = SequentialChain(
    chains=[chain_1,chain_2,chain_3,chain_4],
    input_variables=["text"],
    # Here we return multiple variables
    output_variables=["sentiment","politeness score","negative words","words"],
    verbose=True)


end_time_ms = round(datetime.datetime.now().timestamp() * 1000)  # logged in milliseconds
# text=" The battery is long lasting"
text=["go to hell","the battery is long lasting","hey how are you doing"]
for text in text:

  print(overall_chain({"text":text}))






> Entering new SequentialChain chain...

> Finished chain.
{'text': 'go to hell', 'sentiment': 'negative', 'politeness score': '0', 'negative words': 'go to hell', 'words': 'ten'}


> Entering new SequentialChain chain...

> Finished chain.
{'text': 'the battery is long lasting', 'sentiment': 'positive', 'politeness score': '14%', 'negative words': 'battery', 'words': '2,000'}


> Entering new SequentialChain chain...

> Finished chain.
{'text': 'hey how are you doing', 'sentiment': 'positive', 'politeness score': '80%', 'negative words': 'how are you doing', 'words': '2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
from langchain.llms.base import LLM
import torch

class CustomLLM(LLM):
    model_name = "upstage/llama-65b-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        load_in_8bit=True,
        rope_scaling={"type": "dynamic", "factor": 2}  # allows handling of longer inputs
    )

    def _call(self, prompt, stop=None, **kwargs):
        prompt = f"### User:\n{prompt}\n\n### Assistant:\n"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        output = self.model.generate(**inputs, streamer=self.streamer, use_cache=True, max_new_tokens=float('inf'))
        return self.tokenizer.decode(output[0], skip_special_tokens=True)

    @property
    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=CustomLLM())